In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3



In [5]:
filepath_call = r'C:\Users\User\Nigeria_Telecommunications_Analysis\call_detail_records (1).csv'
df = pd.read_csv(filepath_call)


df.head(10)

,cdr_id,timestamp,operator,calling_number,called_number,call_type,duration_seconds,data_volume_mb,sms_count,tower_id,city,network_type,call_status
0,CDR00000001,2025-09-12 01:47:41,Airtel,2349078978178,2348057135387,data,0,69.63,0,LAG-8972,Lagos,4G,completed
1,CDR00000002,2025-09-14 13:49:36,MTN,2348133004465,2347035891260,data,0,342.29,0,WAR-7779,Warri,4G,completed
2,CDR00000003,2025-09-11 14:09:43,9mobile,2348090322661,2348127029601,voice,2897,0.00,0,BEN-3528,Benin City,3G,failed
3,CDR00000004,2025-09-24 01:05:05,Airtel,2347086040592,2348123412723,voice,1890,0.00,0,LAG-9961,Lagos,2G,completed
4,CDR00000005,2025-09-09 05:12:14,MTN,2348164725206,2349091687278,data,0,435.80,0,WAR-6565,Warri,4G,completed
5,CDR00000006,2025-09-05 12:01:16,Glo,2347058065309,2349058516114,data,0,156.13,0,ABU-9855,Abuja,4G,failed
6,CDR00000007,2025-09-24 13:42:38,Airtel,2347018172609,2347089257065,data,0,421.68,0,LAG-1713,Lagos,4G,dropped
7,CDR00000008,2025-09-28 23:37:25,Glo,2348116235907,2349082663185,voice,1319,0.00,0,ABA-3721,Aba,4G,completed
8,CDR00000009,2025-09-15 12:09:46,Airtel,2347011872192,2349083238788,data,0,24.23,0,KAD-7419,Kaduna,4G,completed
9,CDR00000010,2025-09-15 07:14:28,Glo,2348050193707,2348152000680,data,0,385.45,0,KAN-1861,Kano,2G,completed


In [6]:
def dates_and_time():
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors = 'coerce')
    df['call_date'] = df['timestamp'].dt.date
    df['time'] = df['timestamp'].dt.time
    df.duration_seconds = df.duration_seconds.apply(lambda x: x/60).round(2)



def index_and_sorts():
    df['call_date'] = pd.to_datetime(df['call_date']).dt.date    
    df['peak'] = df['timestamp'].dt.hour
    df.set_index('call_date', inplace= True)
    

dates_and_time()
index_and_sorts()

In [7]:
def error_checking_func(x)-> bool:
    
        a = x['duration_seconds']
        b = x['data_volume_mb']
        c = x['sms_count']


        total = a+b+c
        if (a > 0) + (b>0) + (c>0) == 1:
            return True
        
        else:
            return False




def call_error_checking():
     suspiciuos_calls_df = df[(df['call_type'] == 'voice') & (df['duration_seconds'] > 0) & (df['call_status'] == 'failed')]

     
     ratio = suspiciuos_calls_df.shape[0] / df.shape[0]
     if ratio < 0.05:
         print('Keep')
     else:
          df.drop(suspiciuos_calls_df.index, inplace = True)

call_error_checking()




Keep


In [8]:

def cat_data_check():

    operator_list = [i for i in df.operator.unique()]
    bundle_list = [i for i in df.call_type.unique()]
    city_list = [i for i in df.city.unique()]
    network_list = [i for i in df.network_type.unique()]
    status_list = [i for i in df.call_status.unique()]

    print(f'Operator List is {operator_list}, \n unique bundles are {bundle_list}, \n unique cities are {city_list},  \n all possible networks are {network_list}, \n and all possible statuses are {status_list}' , sep = ':')



cat_data_check()



Operator List is ['Airtel', 'MTN', '9mobile', 'Glo'], 
 unique bundles are ['data', 'voice', 'sms'], 
 unique cities are ['Lagos', 'Warri', 'Benin City', 'Abuja', 'Aba', 'Kaduna', 'Kano', 'Onitsha', 'Ibadan', 'Calabar', 'Ilorin', 'Enugu', 'Jos', 'Port Harcourt', 'Abeokuta'],  
 all possible networks are ['4G', '3G', '2G', '5G'], 
 and all possible statuses are ['completed', 'failed', 'dropped']


In [14]:
df.to_sql('clean_data', sqlite3.connect('nigeria_call_details.db'),if_exists='replace', index=False)

query_one = """
               CREATE TABLE IF NOT EXISTS calling_data_filtered AS
               SELECT
               call_date,
               operator,
               ROUND(duration_seconds / 60.0, 2) AS duration_minutes,
               peak AS call_time_in_hrs
               FROM clean_data
               WHERE call_type = 'voice'
               AND call_status = 'completed';
               """

query_two = """
                    CREATE TABLE IF NOT EXISTS data_usage_at_peak AS
                    SELECT
                    operator,
                    peak,
                    SUM(data_volume_mb) AS total_data_mb
                    FROM clean_data
                    WHERE call_type = 'data'
                    AND (
                    peak BETWEEN 18 AND 23
                    OR peak BETWEEN 12 AND 14
                    )
                    GROUP BY operator, peak;
"""

with sqlite3.connect('nigeria_call_details.db') as conn:
    cursor = conn.cursor()
    cursor.execute(query_one)
    cursor.execute(query_two)
    conn.commit()
